In [4]:
import numpy as np
import pandas as pd
import ast
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import pandas as pd
import json

In [5]:
df_megaset = pd.read_csv('../data/processed_data/megaset.csv')
df_megaset.head()

,Name,Year,Languages,Countries,Genres,Tropes,Budget,domestic_gross,domestic_percentage,foreign_gross,foreign_percentage,worldwide_gross
0,,1916.0,"silent film, hungarian",hungary,"silent film, drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!women art revolution,2010.0,english,"united states of america, canada","lgbt, history, documentary",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#1 cheerleader camp,2010.0,NaN,united states of america,"sports, sex comedy, comedy film, comedy, teen",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#alive,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#horror,2015.0,NaN,NaN,NaN,NaN,1500000.0,NaN,NaN,NaN,NaN,NaN


In [3]:
df_copy_megaset = df_megaset.copy(deep=True)
print(f"Length of megaset {len(df_copy_megaset)}")

Length of megaset 87071


Gonna concatenate all the scraped data.

In [4]:
df_scraped = pd.DataFrame()

length_of_batches = 17

for i in range(length_of_batches):
    s = 'scraped_data/batch' + str(i+1) + '.csv'
    df_temp = pd.read_csv(s, encoding='unicode_escape')
    df_scraped = pd.concat([df_scraped, df_temp])

In [5]:
print(f"The length of df_scraped is {len(df_scraped)}")

The length of df_scraped is 39601


In [6]:
df_scraped.head()

,Name,Year,Domestic,Foreign,Budget,Runtime,Rating,Genre
0,!Women Art Revolution,(2010),"$52,681",NaN,NaN,1 hr 23 min,NaN,Documentary
1,$9.99,(2008),"$52,384","$655,970",NaN,1 hr 18 min,R,Animation;Drama;Fantasy
2,Bedazzled,(1967),NaN,NaN,NaN,1 hr 43 min,NaN,Comedy;Fantasy;Romance
3,'71,(2014),"$1,270,847","$1,791,331",NaN,1 hr 39 min,R,Action;Crime;Drama;Thriller;War
4,Transformers: Rise of the Beasts,(2023),"$157,066,392","$284,314,801",NaN,2 hr 7 min,PG-13,Action;Adventure;Sci-Fi


In [6]:
df_scraped.to_csv('../data/scraped_data/scraped_data_box_offcie_mojo/scraped_data_concatenated.csv', index=False)

NameError: name 'df_scraped' is not defined

Doing the same with thenumbers data

In [8]:
df_scraped_numbers = pd.DataFrame()

length_of_batches = 66

for i in range(length_of_batches):
    if i == 0:
        index = 0
    else:
        index = i*100 + 1
    s = 'numbers_scraped_data/' + str(index) + '.csv'
    df_temp = pd.read_csv(s, encoding='unicode_escape')
    df_scraped_numbers = pd.concat([df_scraped_numbers, df_temp])

In [9]:
df_scraped_numbers.to_csv('../data/numbers_scraped_data/scraped_numbers_concatenated2.csv', index=False)

In [10]:
print(f"The length of the new scraped data is {len(df_scraped_numbers)}")

The length of the new scraped data is 6571


Cleaning up the box_offcie_mojo scraped data


In [11]:
df_scraped['Name'] = df_scraped['Name'].str.lower()
df_scraped['Year'] = df_scraped['Year'].str.replace("(","").str.replace(")","")
df_scraped['Year'] = df_scraped['Year'].astype(int)

In [12]:
df_scraped = df_scraped.drop_duplicates(subset=['Name', 'Year'])
print(len(df_scraped))

27067


Let's take a look at the Megaset. I want to see how much of the cmu dataset is actually left

In [13]:
df_copy_megaset = df_copy_megaset[df_copy_megaset['Year'].notna()]
df_copy_megaset = df_copy_megaset[df_copy_megaset['Name'].notna()]
print(f'Length after dropping NaNs in Year and Name {len(df_copy_megaset)}')

Length after dropping NaNs in Year and Name 80123


In [14]:
df_copy_megaset['Year'] = df_copy_megaset['Year'].astype(int)

In [15]:
df_copy_megaset.head()
df_copy_megaset = df_copy_megaset.drop('Tropes', axis=1)

In [16]:
df_copy_megaset = df_copy_megaset.drop_duplicates(subset=['Name', 'Year'])
print(len(df_copy_megaset))

79650


In [17]:
df_another_megaset_copy = df_copy_megaset.copy(deep=True)
print(len(df_another_megaset_copy))
print(df_another_megaset_copy.isna().sum())
df_another_megaset_copy = df_another_megaset_copy.dropna()
print(len(df_another_megaset_copy))

79650
Name                       0
Year                       0
Languages              14928
Countries              10158
Genres                  6697
Budget                 72814
domestic_gross         76475
domestic_percentage    76703
foreign_gross          76677
foreign_percentage     76703
worldwide_gross        76703
dtype: int64
2112


In [18]:
df_another_megaset_copy = df_another_megaset_copy.drop(df_another_megaset_copy[df_another_megaset_copy.domestic_gross == df_another_megaset_copy.foreign_gross].index)
print(len(df_another_megaset_copy))
df_another_megaset_copy.to_csv('../data/miscellaneous/current_usable_info.csv', index=False)

2112


In [8]:
original_cmu = pd.read_csv('../data/processed_data/cmu_dataset_second_time.csv')
original_cmu.head()

,Name,Year,Revenue,Runtime,Languages,Countries,Genres
0,ghosts of mars,2001,14010832.0,98.0,english,united states of america,"thriller, science fiction, horror, adventure, ..."
1,getting away with murder: the jonbenét ramsey ...,2000,NaN,95.0,english,united states of america,"mystery, biographical film, drama, crime drama"
2,brun bitter,1988,NaN,83.0,norwegian,norway,"crime fiction, drama"
3,white of the eye,1987,NaN,110.0,english,united kingdom,"thriller, erotic thriller, psychological thriller"
4,a woman in flames,1983,NaN,106.0,german,germany,drama


In [20]:
df_checking_from_orginal = pd.merge(original_cmu, df_another_megaset_copy, on=['Name', 'Year'], how='inner')
df_checking_from_orginal = df_checking_from_orginal.drop_duplicates(subset=['Name', 'Year'])
df_checking_from_orginal.head()

,Name,Year,Revenue,Runtime,Languages_x,Countries_x,Genres_x,Languages_y,Countries_y,Genres_y,Budget,domestic_gross,domestic_percentage,foreign_gross,foreign_percentage,worldwide_gross
0,the sorcerer's apprentice,2002,NaN,86.0,english,south africa,"family film, fantasy, adventure, world cinema",english,south africa,"family film, fantasy, adventure, world cinema",160000000.0,63150991.0,0.289702,154835329.0,0.710298,217986320.0
1,becoming jane,2007,37311672.0,112.0,english,"ireland, united kingdom","biography, costume drama, period piece, drama,...",english,"ireland, united kingdom","biography, costume drama, period piece, drama,...",16500000.0,18670946.0,0.474027,20716965.0,0.525973,39387911.0
2,the forbidden kingdom,2008,127906624.0,105.0,"standard mandarin, english, standard cantonese","united states of america, hong kong, china","wuxia, fantasy adventure, adventure, action/ad...","standard mandarin, english, standard cantonese","united states of america, hong kong, china","wuxia, fantasy adventure, adventure, action/ad...",55000000.0,52075270.0,0.403449,77000000.0,0.596551,129075270.0
3,nothing to lose,1997,44480039.0,97.0,english,united states of america,"crime fiction, action/adventure, action, buddy...",english,united states of america,"crime fiction, action/adventure, action, buddy...",25000000.0,44480039.0,0.559222,35059129.0,0.440778,79539168.0
4,resident evil,2002,102441078.0,100.0,english,"united states of america, france, canada, unit...","thriller, science fiction, horror, adventure, ...",english,"united states of america, france, canada, unit...","thriller, science fiction, horror, adventure, ...",35000000.0,40119709.0,0.386557,63667692.0,0.613443,103787401.0


In [21]:
print(f"Length of intersection {len(df_checking_from_orginal)}")

Length of intersection 2112


Damn it's all of them!

In [22]:
print(df_scraped.isna().sum())

Name            0
Year            0
Domestic     1159
Foreign      6777
Budget      23796
Runtime       504
Rating      16959
Genre         277
dtype: int64


In [23]:
df_copy_megaset.head()

,Name,Year,Languages,Countries,Genres,Budget,domestic_gross,domestic_percentage,foreign_gross,foreign_percentage,worldwide_gross
0,,1916,"silent film, hungarian",hungary,"silent film, drama",NaN,NaN,NaN,NaN,NaN,NaN
1,!women art revolution,2010,english,"united states of america, canada","lgbt, history, documentary",NaN,NaN,NaN,NaN,NaN,NaN
2,#1 cheerleader camp,2010,NaN,united states of america,"sports, sex comedy, comedy film, comedy, teen",NaN,NaN,NaN,NaN,NaN,NaN
3,#alive,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#horror,2015,NaN,NaN,NaN,1500000.0,NaN,NaN,NaN,NaN,NaN


Starting the Merge

In [24]:
df_merged = pd.merge(df_copy_megaset, df_scraped, how='outer', on=['Name', 'Year'], suffixes=('_df1', '_df2'))
print(len(df_merged))

91450


In [25]:
df_merged.head()

,Name,Year,Languages,Countries,Genres,Budget_df1,domestic_gross,domestic_percentage,foreign_gross,foreign_percentage,worldwide_gross,Domestic,Foreign,Budget_df2,Runtime,Rating,Genre
0,,1916,"silent film, hungarian",hungary,"silent film, drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!women art revolution,2010,english,"united states of america, canada","lgbt, history, documentary",NaN,NaN,NaN,NaN,NaN,NaN,"$52,681",NaN,NaN,1 hr 23 min,NaN,Documentary
2,#1 cheerleader camp,2010,NaN,united states of america,"sports, sex comedy, comedy film, comedy, teen",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#alive,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#amfad: all my friends are dead,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$156,188","$156,188",NaN,1 hr 31 min,NaN,Horror;Mystery;Thriller


In [26]:
df_merged['Budget'] = df_merged['Budget_df1'].combine_first(df_merged['Budget_df2'])
df_merged['foreign_gross'] = df_merged['foreign_gross'].combine_first(df_merged['Foreign'])
df_merged['domestic_gross'] = df_merged['domestic_gross'].combine_first(df_merged['Domestic'])
df_merged['Genres'] = df_merged['Genres'].combine_first(df_merged['Genre'])

df_merged = df_merged[['Name', 'Year', 'Countries', 'Genres', 'Budget', 'domestic_gross', 'foreign_gross', 'Runtime', 'Rating']]


In [27]:
print(len(df_merged))
df_merged.head()

91450


,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,,1916,hungary,"silent film, drama",NaN,NaN,NaN,NaN,NaN
1,!women art revolution,2010,"united states of america, canada","lgbt, history, documentary",NaN,"$52,681",NaN,1 hr 23 min,NaN
2,#1 cheerleader camp,2010,united states of america,"sports, sex comedy, comedy film, comedy, teen",NaN,NaN,NaN,NaN,NaN
3,#alive,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#amfad: all my friends are dead,2024,NaN,Horror;Mystery;Thriller,NaN,"$156,188","$156,188",1 hr 31 min,NaN


In [28]:
df_merged = df_merged.drop_duplicates(subset=['Name', 'Year'])
df_merged = df_merged[df_merged['Runtime'].notna()]
df_merged = df_merged[df_merged['Rating'].notna()]
print(len(df_merged))

10096


In [29]:
df_merged = df_merged.reset_index(drop=True)
df_merged.head()

,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R
1,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R
2,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13
3,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R
4,'til there was you,1997,united states of america,"romantic comedy, romance film, comedy",NaN,"$3,525,125",NaN,1 hr 53 min,PG-13


In [30]:
rows_to_drop = df_merged[df_merged['domestic_gross'] == df_merged['foreign_gross']].index
df_merged = df_merged.drop(index=rows_to_drop)
print(len(df_merged))

8816


In [31]:
df_merged.head()

,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R
1,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R
2,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13
3,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R
4,'til there was you,1997,united states of america,"romantic comedy, romance film, comedy",NaN,"$3,525,125",NaN,1 hr 53 min,PG-13


In [32]:
df_merged.isnull().sum()

Name                 0
Year                 0
Countries         3609
Genres               0
Budget            4645
domestic_gross     187
foreign_gross     2530
Runtime              0
Rating               0
dtype: int64

In [9]:
df_wiki_scrape = pd.read_csv('../data/scraped_data/wiki_extracted.csv')
df_wiki_scrape = df_wiki_scrape.drop(['Unnamed: 0'], axis=1)
df_wiki_scrape.head()
print(len(df_wiki_scrape))

455


In [34]:
df_merged_wiki = pd.merge(df_merged, df_wiki_scrape, how='outer', on=['Name', 'Year'], suffixes=('_df1', '_df2'))

In [35]:
print(len(df_merged_wiki))
df_merged_wiki.head()

8862


,Name,Year,Countries_df1,Genres_df1,Budget_df1,domestic_gross_df1,foreign_gross_df1,Runtime,Rating,Languages,Countries_df2,Genres_df2,Budget_df2,domestic_gross_df2,domestic_percentage,foreign_gross_df2,foreign_percentage,worldwide_gross
0,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'til there was you,1997,united states of america,"romantic comedy, romance film, comedy",NaN,"$3,525,125",NaN,1 hr 53 min,PG-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_merged_wiki['Budget'] = df_merged_wiki['Budget_df1'].combine_first(df_merged_wiki['Budget_df2'])
df_merged_wiki['foreign_gross'] = df_merged_wiki['foreign_gross_df1'].combine_first(df_merged_wiki['foreign_gross_df2'])
df_merged_wiki['domestic_gross'] = df_merged_wiki['domestic_gross_df1'].combine_first(df_merged_wiki['domestic_gross_df2'])
df_merged_wiki['Genres'] = df_merged_wiki['Genres_df1'].combine_first(df_merged_wiki['Genres_df2'])
df_merged_wiki['Countries'] = df_merged_wiki['Countries_df1'].combine_first(df_merged_wiki['Countries_df2'])

df_merged_wiki = df_merged_wiki[['Name', 'Year', 'Countries', 'Genres', 'Budget', 'domestic_gross', 'foreign_gross', 'Runtime', 'Rating']]


In [37]:
df_merged_wiki = df_merged_wiki[df_merged_wiki['Runtime'].notna()]
df_merged_wiki = df_merged_wiki[df_merged_wiki['Rating'].notna()]
print(len(df_merged_wiki))
df_merged_wiki.head()

8816


,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R
1,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R
2,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13
3,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R
4,'til there was you,1997,united states of america,"romantic comedy, romance film, comedy",NaN,"$3,525,125",NaN,1 hr 53 min,PG-13


In [38]:
df_merged_wiki.isnull().sum()

Name                 0
Year                 0
Countries         3208
Genres               0
Budget            4645
domestic_gross     187
foreign_gross     2530
Runtime              0
Rating               0
dtype: int64

Manually check scraped_numbers_concatenated and replace weird symbols

In [10]:
df_the_numbers = pd.read_csv('../data/scraped_data/numbers_scraped_data/scraped_numbers_concatenated.csv')

Some of this data was manually corrected

In [123]:
def remove_dolla_and_comma(s):
    return float(str(s).replace('$', '').replace(',',''))

def make_genres_right(s):
    return str(s).replace(';', ', ').lower()

def runtime(s):
    new_s = str(s).split()
    min_h = 0
    min_m = 0
    flag = False
    for i in range(len(new_s)):
        if new_s[i] == 'h' or new_s[i] == 'hr':
            min_h = float(new_s[i-1])*60
            flag = True
        if new_s[i] == 'min' or new_s[i] == 'mins':
            min_m = float(new_s[i-1])
            flag = True
    if flag:
        return int(min_h + min_m)
    else:
        return s
    

In [41]:
df_merged_wiki.head()

,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R
1,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R
2,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13
3,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R
4,'til there was you,1997,united states of america,"romantic comedy, romance film, comedy",NaN,"$3,525,125",NaN,1 hr 53 min,PG-13


In [42]:
def remove_dolla_and_comma(s):
    if not pd.isna(s):
        new_s=re.sub("\(.*\)","",s) 
        return float(str(new_s).replace('$', '').replace(',',''))

def remove_dolla_and_comma_v2(s, test=False):
    if not pd.isna(s):
        if test == True:
            #print(s)
            #print(float(str(s).replace('$', '').replace(',','')))
            return int(float(str(s).replace('$', '').replace(',','')))
        return float(str(s).replace('$', '').replace(',',''))

In [43]:
df_the_numbers['Name'] = df_the_numbers['Name'].apply(lambda x: str(x).lower())
df_the_numbers['Languages'] = df_the_numbers['Languages'].apply(lambda x: str(x).lower())
df_the_numbers['Countries'] = df_the_numbers['Countries'].apply(lambda x: str(x).lower())
df_the_numbers['Genres'] = df_the_numbers['Genres'].apply(lambda x: str(x).lower())
df_the_numbers.rename(columns={'runtime':'Runtime', 'rating':'Rating'}, inplace=True)

In [44]:
cols = df_the_numbers.select_dtypes(['object']).columns
df_the_numbers[cols] = df_the_numbers[cols].apply(lambda x: x.str.strip())

In [45]:
df_the_numbers = df_the_numbers[df_the_numbers['Year'].notna()]
df_the_numbers = df_the_numbers[df_the_numbers['Name'].notna()]
df_the_numbers['Year'] = df_the_numbers['Year'].astype(int)

In [46]:
print(len(df_the_numbers))

6561


In [47]:
import re
df_the_numbers = df_the_numbers[df_the_numbers['Budget'].notna()]
print(len(df_the_numbers))
df_the_numbers['Budget'] = df_the_numbers['Budget'].apply(lambda x: remove_dolla_and_comma(x))
#df_the_numbers['Budget'] = df_the_numbers['Budget'].astype(int)

6552


In [48]:
df_the_numbers['domestic_gross'] = df_the_numbers['domestic_gross'].apply(lambda x: remove_dolla_and_comma_v2(x))
#df_the_numbers['domestic_gross'] = df_the_numbers['domestic_gross'].astype(int)

df_the_numbers['foreign_gross'] = df_the_numbers['foreign_gross'].apply(lambda x: remove_dolla_and_comma_v2(x))
#df_the_numbers['foreign_gross'] = df_the_numbers['foreign_gross'].astype(int)

df_the_numbers['worldwide_gross'] = df_the_numbers['worldwide_gross'].apply(lambda x: remove_dolla_and_comma_v2(x, True))
#df_the_numbers['worldwide_gross'] = df_the_numbers['worldwide_gross'].astype(int)
#df_the_numbers['Year'] = df_the_numbers['Year'].astype(int)
df_the_numbers['Rating'] = df_the_numbers['Rating'].str.split(' for').str[0]
df_the_numbers['Runtime'] = df_the_numbers['Runtime'].str.split(' minutes').str[0]
df_the_numbers['Countries'] = df_the_numbers['Countries'].replace('united states','united states of america', regex=True)

In [49]:
df_the_numbers.head()

,Name,Year,Languages,Countries,Genres,Budget,Runtime,Rating,domestic_gross,foreign_gross,worldwide_gross
0,star wars ep. vii: the force awakens,2015,english,united states of america,adventure,533200000.0,136,PG-13,936662225.0,1.119385e+09,2.056047e+09
1,avatar: the way of water,2022,english,united states of america,action,460000000.0,190,PG-13,684075767.0,1.633439e+09,2.317514e+09
2,indiana jones and the dial of destiny,2023,english,united states of america,adventure,402300000.0,142,PG-13,174480468.0,2.094826e+08,3.839631e+08
3,avengers: endgame,2019,english,united states of america,action,400000000.0,181,PG-13,858373000.0,1.889870e+09,2.748243e+09
4,pirates of the caribbean: on stranger tides,2011,english,united states of america,adventure,379000000.0,136,PG-13,241071802.0,8.046420e+08,1.045714e+09


In [50]:
df_merged_numbers = pd.merge(df_merged_wiki, df_the_numbers, how='outer', on=['Name', 'Year'], suffixes=('_df1', '_df2'))

In [51]:
print(len(df_merged_numbers))
df_merged_numbers.head()

11610


,Name,Year,Countries_df1,Genres_df1,Budget_df1,domestic_gross_df1,foreign_gross_df1,Runtime_df1,Rating_df1,Languages,Countries_df2,Genres_df2,Budget_df2,Runtime_df2,Rating_df2,domestic_gross_df2,foreign_gross_df2,worldwide_gross
0,#horror,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,english,united states of america,horror,1500000.0,90,Not Rated,NaN,NaN,NaN
1,$9.99,2008,"australia, israel","stop motion, drama, indie, world cinema, anima...",NaN,"$52,384","$655,970",1 hr 18 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,'71,2014,NaN,Action;Crime;Drama;Thriller;War,NaN,"$1,270,847","$1,791,331",1 hr 39 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"'night, mother",1986,united states of america,"family drama, drama",NaN,"$441,863",NaN,1 hr 36 min,PG-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'r xmas,2001,"france, united states of america","crime fiction, crime thriller, drama, thriller...",NaN,$850,"$98,230",1 hr 25 min,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df_merged_numbers['Budget'] = df_merged_numbers['Budget_df1'].combine_first(df_merged_numbers['Budget_df2'])
df_merged_numbers['foreign_gross'] = df_merged_numbers['foreign_gross_df1'].combine_first(df_merged_numbers['foreign_gross_df2'])
df_merged_numbers['domestic_gross'] = df_merged_numbers['domestic_gross_df1'].combine_first(df_merged_numbers['domestic_gross_df2'])
df_merged_numbers['Genres'] = df_merged_numbers['Genres_df1'].combine_first(df_merged_numbers['Genres_df2'])
df_merged_numbers['Countries'] = df_merged_numbers['Countries_df1'].combine_first(df_merged_numbers['Countries_df2'])
df_merged_numbers['Runtime'] = df_merged_numbers['Runtime_df1'].combine_first(df_merged_numbers['Runtime_df2'])
df_merged_numbers['Rating'] = df_merged_numbers['Rating_df1'].combine_first(df_merged_numbers['Rating_df2'])

df_merged_numbers = df_merged_numbers[['Name', 'Year', 'Countries', 'Genres', 'Budget', 'domestic_gross', 'foreign_gross', 'Runtime', 'Rating']]


In [53]:
rows_to_drop = df_merged_numbers[df_merged_numbers['domestic_gross'] == df_merged_numbers['foreign_gross']].index
df_merged_numbers = df_merged_numbers.drop(index=rows_to_drop)
print(len(df_merged_numbers))

11607


In [54]:
df_merged_numbers = df_merged_numbers.dropna()
print(len(df_merged_numbers))
df_merged_numbers.head()

5168


,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
7,10 cloverfield lane,2016,united states,Drama;Horror;Sci-Fi;Thriller,5000000.0,72082999.0,38134000.0,1 hr 43 min,PG-13
10,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000.0,"$38,178,166","$15,334,203",1 hr 37 min,PG-13
12,"10,000 b.c.",2008,united states of america,adventure,105000000.0,94784201.0,174281477.0,109,PG-13
13,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...","$105,000,000","$94,784,201","$175,000,000",1 hr 49 min,PG-13
16,10th & wolf,2006,nan,drama,8000000.0,54702.0,89080.0,107,Not Rated


In [55]:
df_merged_numbers = df_merged_numbers.drop_duplicates()
print(len(df_merged_numbers))

5156


In [56]:
df_merged_numbers.to_csv("../data/miscellaneous/better_cleaned.csv", index=False)

In [57]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

vc = df_merged_numbers['Name'].value_counts()
pd.options.display.max_seq_items = 1000
print(vc[vc > 1].index)


Index(['halloween', 'crash', 'pinocchio', 'king kong', 'scream', 'case 39',
       'casablanca', 'carrie', 'jeff, who lives at home', 'serenity',
       'cabin fever', 'but i'm a cheerleader', 'bug', 'buffalo soldiers',
       'bloodrayne', 'killer joe', 'casino royale', 'brother',
       'brooklyn's finest', 'brick lane', 'the square', 'bon voyage',
       'the texas chainsaw massacre', 'rush', 'the three musketeers',
       'brothers', 'shade', 'sexy beast', 'the perfect game', 'impostor',
       'the ninth gate', 'sleight', 'the omen', 'the oxford murders',
       'clash of the titans', 'silent house', 'inkheart', 'city island',
       'the perfect host', 'shanghai', 'cinderella', 'chloe',
       'the place beyond the pines', 'invincible', 'charlie bartlett',
       'the promise', 'the puffy chair', 'shaolin soccer', 'the railway man',
       'rubber', 'movie 43', 'smokin' aces', 'kingsman: the secret service',
       'transamerica', 'psycho', 'true grit', 'tsotsi', 'twilight',
    

Manually check some of the data and fill some missing values

In [190]:
df_cleaned_manual = pd.read_csv("../data/miscellaneous/better_cleaned_manual_clean.csv", encoding='latin-1')

In [191]:
df_cleaned_manual = df_cleaned_manual.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\lolon\AppData\Local\Temp\ipykernel_16928\195832297.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned_manual = df_cleaned_manual.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [192]:
df_cleaned_manual['Year'] = df_cleaned_manual['Year'].astype(int)

In [193]:
print(len(df_cleaned_manual))
df_cleaned_manual.head()

5069


,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,10 cloverfield lane,2016,united states,drama;horror;sci-fi;thriller,5000000.0,72082999.0,38134000.0,1 hr 43 min,pg-13
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000.0,"$38,178,166","$15,334,203",1 hr 37 min,pg-13
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...","$105,000,000","$94,784,201","$175,000,000",1 hr 49 min,pg-13
3,12 monkeys,1995,united states,mystery;sci-fi;thriller,29000000.0,"$57,141,459","$111,698,000",2 hr 9 min,r
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000.0,"$12,234,694","$5,045,632",1 hr 48 min,pg-13


In [194]:
indices = []
for index, row in df_cleaned_manual.iterrows():
    if index == len(df_cleaned_manual) - 1: continue
    if row['Name'] == df_cleaned_manual.loc[index+1, 'Name']:
        if abs(row['Year'] - df_cleaned_manual.loc[index+1, 'Year']) < 2:
            indices.append(index+1)
print(indices)

[43, 64, 96, 848, 900, 935, 970, 1056, 1059, 1082, 1132, 1144, 1175, 1211, 1231, 1309, 1358, 1417, 1450, 1462, 1613, 1623, 1629, 1723, 2385, 2397, 2417, 2436, 2488, 2518, 2536, 2595, 2631, 2674, 2694, 2713, 2718, 2721, 2739, 2748, 3378, 3410, 3415, 3429, 3479, 3508, 3523, 3529, 3573, 3611, 3664, 3670, 3688, 3690, 3693, 3746, 3752, 3758, 3790, 3797, 3838, 3845, 3855, 4031, 4061, 4065, 4074, 4084, 4100, 4152, 4169, 4881, 4890, 4908, 4974, 5049]


In [195]:
df_cleaned_manual = df_cleaned_manual.drop(index=indices)
print(len(df_cleaned_manual))

4993


In [196]:
df_cleaned_manual['Budget'] = df_cleaned_manual['Budget'].apply(lambda x: remove_dolla_and_comma(str(x)))
df_cleaned_manual['Budget'] = df_cleaned_manual['Budget'].astype(int)

df_cleaned_manual['domestic_gross'] = df_cleaned_manual['domestic_gross'].apply(lambda x: remove_dolla_and_comma(str(x)))
df_cleaned_manual['domestic_gross'] = df_cleaned_manual['domestic_gross'].astype(int)

df_cleaned_manual['foreign_gross'] = df_cleaned_manual['foreign_gross'].apply(lambda x: remove_dolla_and_comma(str(x)))
df_cleaned_manual['foreign_gross'] = df_cleaned_manual['foreign_gross'].astype(int)

df_cleaned_manual['Runtime'] = df_cleaned_manual['Runtime'].apply(lambda x: runtime(str(x)))
df_cleaned_manual['Runtime'] = df_cleaned_manual['Runtime'].astype(int)

df_cleaned_manual['Genres'] = df_cleaned_manual['Genres'].apply(lambda x: make_genres_right(str(x)))


In [197]:
df_cleaned_manual.head()

,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,38134000,103,pg-13
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,15334203,97,pg-13
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,175000000,109,pg-13
3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,111698000,129,r
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,5045632,108,pg-13


In [198]:
df_cleaned_manual['Rating'].value_counts()

Rating
r                                                                                                                                        2163
pg-13                                                                                                                                    1760
pg                                                                                                                                        767
not rated                                                                                                                                 138
g                                                                                                                                          97
nc-17                                                                                                                                      15
r edited                                                                                                                                    3

In [199]:
def check_valid_rating(s):
    valid_ratings = ['pg-13', 'nc-17', 'gp', 'pg', 'g', 'r']
    if "not" in s: return "remove"
    for i in range(len(valid_ratings)):
        if valid_ratings[i] in s[:5]:
            return valid_ratings[i]
    return "remove"
        
df_cleaned_manual['Rating'] = df_cleaned_manual['Rating'].apply(lambda x: check_valid_rating(str(x)))


In [200]:
df_cleaned_manual['Rating'].value_counts()

Rating
r         2176
pg-13     1766
pg         777
remove     146
g          111
nc-17       15
gp           2
Name: count, dtype: int64

In [201]:
remove_indices = df_cleaned_manual[df_cleaned_manual['Rating'] == "remove"].index
df_cleaned_manual = df_cleaned_manual.drop(index = remove_indices)
df_cleaned_manual = df_cleaned_manual.reset_index(drop=True)
print(len(df_cleaned_manual))
df_cleaned_manual.head()

4847


,Name,Year,Countries,Genres,Budget,domestic_gross,foreign_gross,Runtime,Rating
0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,38134000,103,pg-13
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,15334203,97,pg-13
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,175000000,109,pg-13
3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,111698000,129,r
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,5045632,108,pg-13


In [202]:
df_cleaned_manual = df_cleaned_manual.rename(columns={'domestic_gross': 'Domestic', 'foreign_gross': 'Foreign'})

In [203]:
df_cleaned_manual.head()

,Name,Year,Countries,Genres,Budget,Domestic,Foreign,Runtime,Rating
0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,38134000,103,pg-13
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,15334203,97,pg-13
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,175000000,109,pg-13
3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,111698000,129,r
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,5045632,108,pg-13


In [207]:
df_cleaned_manual['Worldwide'] = df_cleaned_manual['Worldwide'].astype('int64')
df_cleaned_manual['Worldwide'] = df_cleaned_manual['Domestic'] + df_cleaned_manual['Foreign']
df_cleaned_manual['Domestic_Percentage'] = df_cleaned_manual['Domestic'] / df_cleaned_manual['Worldwide']
df_cleaned_manual['Foreign_Percentage'] = df_cleaned_manual['Foreign'] / df_cleaned_manual['Worldwide']

In [208]:
df_cleaned_manual.head()

,Name,Year,Countries,Genres,Budget,Domestic,Foreign,Runtime,Rating,Worldwide,Domestic_Percentage,Foreign_Percentage
0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,38134000,103,pg-13,110216999,0.654010,0.345990
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,15334203,97,pg-13,53512369,0.713446,0.286554
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,175000000,109,pg-13,269784201,0.351333,0.648667
3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,111698000,129,r,168839459,0.338437,0.661563
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,5045632,108,pg-13,17280326,0.708013,0.291987


In [213]:
df_cleaned_manual = df_cleaned_manual[['Name', 'Year', 'Countries', 'Genres', 'Budget', 'Domestic', 'Domestic_Percentage', 'Foreign', 'Foreign_Percentage', 'Worldwide', 'Runtime', 'Rating']]
df_cleaned_manual = df_cleaned_manual.rename(columns={'Budget': 'Budget(USD)', 'Domestic': 'Domestic(USD)', 'Foreign':'Foreign(USD)', 'Worldwide':'Worldwide(USD)', 'Runtime':'Runtime(mins)'})
df_cleaned_manual.head()

,Name,Year,Countries,Genres,Budget(USD),Domestic(USD),Domestic_Percentage,Foreign(USD),Foreign_Percentage,Worldwide(USD),Runtime(mins),Rating
0,10 cloverfield lane,2016,united states,"drama, horror, sci-fi, thriller",5000000,72082999,0.654010,38134000,0.345990,110216999,103,pg-13
1,10 things i hate about you,1999,united states of america,"romantic comedy, romance film, coming of age, ...",13000000,38178166,0.713446,15334203,0.286554,53512369,97,pg-13
2,"10,000 bc",2008,"united states of america, south africa, new ze...","thriller, adventure, costume drama, action/adv...",105000000,94784201,0.351333,175000000,0.648667,269784201,109,pg-13
3,12 monkeys,1995,united states,"mystery, sci-fi, thriller",29000000,57141459,0.338437,111698000,0.661563,168839459,129,r
4,12 rounds,2009,united states of america,"action thrillers, thriller, action, crime fiction",20000000,12234694,0.708013,5045632,0.291987,17280326,108,pg-13


In [216]:
df_cleaned_manual.to_csv('../data/miscellaneous/almost_final_cleaned.csv', index=False)